[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap03/cap03-main.ipynb)

# Capítulo 3 - MDPs, Retornos e Funções de Valor


In [ ]:
import gym
import numpy as np

In [ ]:
# vamos focar nesses três ambientes por serem mais simples
#env = gym.make("MountainCar-v0")
#env = gym.make("CartPole-v1")
#env = gym.make("Taxi-v3")
env = gym.make("FrozenLake-v1")

![Figura mostrando interação agente(política)-ambiente](figura_mdp.png "Interação agente-ambiente")

# 1 - Episódio e Trajetória

Um *episódio* é uma execução da tarefa (ou do ambiente gym). 

E a *trajetória* é a sequência de estados (observações), ações e recompensas do episódio. Assumindo um episódio de $n$ passos (ações aplicadas):

$S_0 \rightarrow A_0 \rightarrow R_1 \rightarrow S_1 \rightarrow A_1 \rightarrow R_2 \rightarrow S_2 \rightarrow \cdots S_{n-1} \rightarrow A_{n-1} \rightarrow R_n \rightarrow S_n$

Vamos ilustrar um episódio em um MDP usando o ambiente *"env"* escolhido no código acima. 

Estamos assumindo que o episódio encerrou de fato (chegou em um estado final) em *$n$="TOTAL_STEPS"* passos.

In [ ]:
TOTAL_STEPS = 5

i = 0
obs = env.reset()
print(f"S0 = {obs}")

done = False

# roda apenas alguns passos
for i in range(0,TOTAL_STEPS):
    #env.render()
    action = env.action_space.sample()
    print(f" A{i} = {action}")

    next_obs, reward, done, info = env.step(action)

    print(f"  R{i+1} = {reward}")
    print(f"S{i+1} = {next_obs}")

    obs = next_obs
    #time.sleep(0.1)

env.close()

Os detalhes do *episódio* que mostramos acima são chamamos de *trajectory* ou *rollout*.

Dependendo do algoritmo, vamos precisar analisar essas informações em trios (S,A,R) ou quádruplas (S,A,R,S) ou até quíntuplas (S,A,R,S',A').

Abaixo, vamos agrupar e guardar em trio os dados de 1 episódio completo.

In [ ]:
obs = env.reset()
trajectory = [] 

done = False

while not done:
    action = env.action_space.sample()
    next_obs, reward, done, _ = env.step(action)
    trajectory.append( (obs, action, reward) )
    obs = next_obs

env.close()

print("Trajetória como sequência de trios (STATE, ACTION, REWARD):")
trajectory

A escolha de uma ação a cada estado é chamada de **política**. Até aqui, estamos usando uma política aleatória (que escolhe qualquer das ações disponíveis com igual probabilidade). 

O nosso objetivo é ver os algoritmos que aprendem uma política "boa". Mas ainda não veremos nesta aula.

Para o restante deste notebook, vamos usar uma política simples que chamamos de **policy_0**. Ela:
- com 50% de chance, escolhe a ação **0**
- com 5o% de chance, escolhe qualquer ação aleatoriamente

Veja o código dela:

In [ ]:
num_actions = env.action_space.n

def policy0(obs):
    x = np.random.random()
    if x <= 0.5:
        return 0
    else:
        return np.random.randint(1, num_actions)

Também vamos definir uma função `run_episode()` para gerar uma trajetória (de 1 episódio completo) usando uma política dada como parâmetro:

In [ ]:
def run_episode(env, agent_policy):
    obs = env.reset()
    trajectory = [] 

    done = False

    while not done:
        action = agent_policy(obs)
        next_obs, reward, done, _ = env.step(action)
        trajectory.append( (obs, action, reward) )
        obs = next_obs
    
    env.close()
    
    #trajectory.append( (obs, None, None) )
    return trajectory

Agora, vamos criar uma trajetória com a política proposta antes:

In [ ]:
print("Trajetória:")
trajectory = run_episode(env, agent_policy=policy0)
print(trajectory)

# 2 - Calcular os Retornos

O *retorno (final)* $G$ é uma medida da recompensa total obtida ao longo de um episódio. 

Em um MDP, o objetivo é otimizar o valor médio de $G$, para infinitos episódios.


### 2.1 - Retorno final do episódio ($G$)



Para um episódio com $n$ passos, o **retorno (não-descontado)** é calculado assim:

$ G = R_1 + R_2 + R_3 + \cdots + R_n = \displaystyle\sum_{i=1}^{n} R_i$

No código a seguir, vamos calcular o *retorno não-descontado* da trajetória calculada antes.

*Observação*: Em código, como "return" é uma palavra reservada de Python, o *retorno* do episódio será representando por nomes como:
- `sum_rewards`
- ou `episode_return`
- ou `episode_reward` 
- ou versões abreviadas desses nomes.


In [ ]:
sum_rewards = 0.0
for (s, a, r) in trajectory:
    sum_rewards += r

print("Retorno não-descontado:", sum_rewards)

Porém, é mais usado o **retorno descontado** de um episódio.

Neste caso, $G$ é uma soma que "atenua" recompensas mais distantes, valorizando mais as recompensas iniciais. (Você prefere receber 100 reais agora, de uma vez, ou em 100 parcelas de 1 real?)

Para isso, a cada passo, a recompensa tem uma *redução* na sua relevância, dada por um parâmetro $\gamma\;$, tal que $0 < \gamma \leq 1$.

Para um episódio com $n$ passos, o *retorno descontado* é calculado assim:

$ G = R_1 + \gamma R_2 + \gamma^2 R_3 + \cdots + \gamma^{(n-1)} R_n = \displaystyle\sum_{i=1}^{n} \gamma^{(i-1)} R_i$

Vamos criar uma função para fazer esse cálculo, a partir de uma dada trajetória (de 1 episódio):

In [ ]:
def get_episode_return(trajectory, gamma):
    step = 0
    discounted_sum_rewards = 0.0
    for (s, a, r) in trajectory:
        discounted_sum_rewards += (gamma ** step) * r
        step += 1
    return discounted_sum_rewards

A seguir, calculamos o *retorno descontado* da trajetória calculada na seção anterior, assumindo um valor específico de $\gamma$ (variável `GAMMA`):

In [ ]:
GAMMA = 0.99

epi_return = get_episode_return(trajectory, gamma=GAMMA)
print("Retorno descontado:", epi_return)

### 2.2 - Retornos intermediários a cada passo ($G_i$)



Também podemos calcular um retorno parcial, a partir de um passo específico $i$ de um dado episódio:

$$
\begin{align*}
   && &\quad G_0     &= &\;\;R_1 &+ \gamma &R_2 &+ \gamma^2 &R_3 &+ \gamma^3 &R_4 &+ \gamma^4 &R_5 &+ \;\cdots \;&+ \gamma^{n-1} &R_n & \;\;\;(= G) \\
   && &\quad G_1     &= &        &         &R_2 &+ \gamma   &R_3 &+ \gamma^2 &R_4 &+ \gamma^3 &R_5 &+ \;\cdots \;&+ \gamma^{n-2} &R_n &       \\ 
   && &\quad G_2     &= &        &         &    &           &R_3 &+ \gamma   &R_4 &+ \gamma^2 &R_5 &+ \;\cdots \;&+ \gamma^{n-3} &R_n &       \\
   && &\quad G_3     &= &        &         &    &           &    &           &R_4 &+ \gamma   &R_5 &+ \;\cdots \;&+ \gamma^{n-4} &R_n &       \\
   && &\quad         &\cdots   & & & & & & & & & & & &    & \\
   && &\quad G_{n-1} &= &        & & & & & & & & & & &R_n & \\
   && &\quad G_n     &= &\;\;0 & & & & & & & & & & &      & \\
\end{align*}
$$

Podemos calcular um retorno parcial $G_i$ simplesmente omitindo os $i$ passos inicias da trajectória:

In [ ]:
for i in range(len(trajectory)+1):
    Gi = get_episode_return(trajectory[i:], GAMMA)
    print(f"Retorno parcial G_{i} :", Gi)

Observe novamente a série de equações anteriores para os retornos parciais $G_0$, $G_1$, $G_2$, etc.

Percebe que existe apenas uma pequena mudança entre cada equação (para $G_i$) e a equação logo abaixo?

De fato, existe uma relação (recursiva) entre $G_i$ e $G_{i+1}$ que pode ser expressa assim:
$$
   G_{i} = R_{i+1} + \gamma G_{i+1}
$$

Usando essa relação recursiva, podemos calcular todos os retornos parciais de maneira mais simples:

In [ ]:
# calcula os retornos parciais a partir de cada passo 
# em ordem invertida (G_i, para cada i de n a 0) 
i = len(trajectory)

Gi = 0.0
print(f"G_{i} =", Gi)

for (s, a, r) in reversed(trajectory):   
    i = i - 1
    Gi = r + GAMMA*Gi
    print(f"G_{i} =", Gi)


# 3 - Funções de Valor

São funções que não fazem parte da essência de um MDP, mas são úteis para criar algoritmos.

Todas elas fazem avaliações dos *retornos esperados* (médios) para uma *política específica*! 

Veremos dois tipos de função de valor, a seguir. Ambas são calculadas a partir dos retornos intermediários.


### 3.1 Função de valor do estado $V(s)$

Esta função dá o retorno esperado a partir de cada estado $s$, para uma política específica.

De forma matemática, ela é definida assim:

$$V(s) = E[G_t | S_t=s]$$ 

#### Visão Algorítmica

Uma explicação mais algorítmica do valor de $V(s)$ para um $s$ específico é dada a seguir:

---
1. rode infinitos episódios com a política
2. analise cada episódio, e a cada passo iniciado no estado $s_i$:
   - calcule o retorno parcial $G_i$
   - salve $G_i$ no histórico do estado $s_i$
3. Para cada estado $s$:
   - $V(s)$ = média de todos os retornos do histórico do estado $s$

---

Implementamos esta ideia abaixo, rodando 5000 episódios. Para isso, vamos anexar cada retorno a um dicionário `returns_history` (um histórico dos retornos) indexado pelo $s$ onde se originou o retorno intermediário.

Esta implementação assume ambiente de *estado discreto*, representados por inteiros iniciados em 0. Exemplos de ambientes assim são *FrozenLake* e *Taxi*.

In [ ]:
# associa cada estado a uma lista de retornos parciais 
# obtidos a partir do estado (em um episódio qualquer)
returns_history = dict()

# roda muitos episódios, com a política desejada
for epi in range(5000):
    trajectory = run_episode(env, policy0)
    
    # calcula os retornos a cada passo (G_i, para cada i de n a 0) do episódio
    # guardando o valor em returns_history
    Gi = 0.0
    for (s, a, r) in reversed(trajectory):   
        Gi = r + GAMMA*Gi
        if s not in returns_history.keys():
            returns_history[s] = [ Gi ]
        else:
            returns_history[s].append(Gi)

# associa cada estado à média dos retornos parciais
V = np.zeros(env.observation_space.n)

# calcula V
for s in returns_history.keys():
    V[s] = np.mean( returns_history[s] )

V

Note que o $V$ calculado para estados discretos (que são representados por inteiros) pode ser representado como um array. O estado é usado como o índice para acessar o array, onde está guardado o valor daquele estado.

### 3.2 Função de valor do estado-ação Q(s,a)

De maneira análoga ao $V(s)$, o $Q(s,a)$ representa o retorno esperado a partir do par $(s,a)$:

$$Q(s,a) = E[G_t | S_t=s, A_t=a]$$ 

Em outras palavras, $Q(s,a)$ responde a esta pergunta:

*"Quando estava no estado **s** e fez a ação **a**, qual o retorno esperado (se continuar seguindo a política no restante do episódio)?"*

A definição e a forma de calcular é análoga ao $V(s)$, mas vamos usar um dicionário `returns_history` indexado pelo par $(s,a)$ onde se originou cada retorno $G_i$.

In [ ]:
# associa cada para (estado, ação) a uma lista de retornos parciais 
# obtidos a partir do estado ao realizar aquela ação (em um episódio qualquer)
returns_history = dict()

# roda muitos episódios, com a política desejada
for epi in range(5000):
    trajectory = run_episode(env, policy0)

    # calcula os retornos a cada passo (G_i, para cada i de n a 0) do episódio
    # guardando o valor em returns_history

    # completar...
    

# matriz para associar cada par (estado, ação) à média dos retornos parciais
Q = np.zeros(shape=(env.observation_space.n, env.action_space.n))

# calcula Q

# completar...

Note que a função $Q$ calculado para estados discretos e ações discretas (ambos representados por inteiros) pode ser representada como uma matriz. Ela pode ser vista como uma _tabela_ com linhas representando os *estados* e colunas representando as *ações*. 

Por esse motivo, chamamos essa representação de **Q-table** (tabela-Q).

# 4 - Preparando para os Métodos Baseados em Q-table

Agora sim, podemos falar um pouquinho sobre os algoritmos de controle para RL, que são os algoritmos capazes de aprender políticas "boas", ou seja, políticas que dão altos retornos.

Coloque-se no lugar do algoritmo e suponha que você inicie com uma política qualquer (provavelmente ruim) e que você tenha calculado o *Q* dessa política com uma Q-table.

**De que forma você poderia melhorar a política olhando para os valores de Q?**

**Ou, como escolher a melhor ação a cada estado, usando a Q-table?**

---

No próxima parte do curso, veremos um algoritmo para RL, da família Monte-Carlo, onde a política é implicitamente representada pelo $Q$. 

Este método repete $N$ vezes esses passos:
1. Rode um episódio, usando a política representada pela tabela $Q$
   - salve a trajetória
1. Depois, calcule os valores de $G_i$ e use esses valores para atualizar $Q$
   - ao atualizar $Q$, a política também muda

---

Veremos mais detalhes no próximo capítulo.